<a href="https://colab.research.google.com/github/a-gasior/Sarcasm-Detection/blob/master/Sarcasm_Detection_Using_Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Andrew Gasiorowski

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import re as _re
import pandas as _pd
import numpy as np
import json as _js
from math import log as _log
from sklearn.model_selection import StratifiedKFold, train_test_split
from collections import Counter
import pickle
from operator import itemgetter
import matplotlib.pyplot as _plt

In [0]:
def _get_pickles():
  pickle_in = open("gdrive/My Drive/Data/pickles/reddit_training_x.pickle","rb")
  x_train = pickle.load(pickle_in)
  
  pickle_in = open("gdrive/My Drive/Data/pickles/reddit_training_y.pickle","rb")
  y_train = pickle.load(pickle_in)
  
  pickle_in = open("gdrive/My Drive/Data/pickles/reddit_testing_x.pickle","rb")
  x_test = pickle.load(pickle_in)
  
  pickle_in = open("gdrive/My Drive/Data/pickles/reddit_testing_y.pickle","rb")
  y_test = pickle.load(pickle_in)
  
  return x_train, y_train, x_test, y_test 

In [0]:
def _extract_uni_grams(x):
  new_list = []
  for idx,sentence in enumerate(x):
    uni_gram = _re.findall(r'(?=\b(\w+)\b)', sentence)
    new_list.append(uni_gram)
  return _pd.Series(new_list)


def _extract_bi_grams(x):
  new_list = []
  for idx,sentence in enumerate(x):
    bi_gram = _re.findall(r'(?=\b(\w+)\b\S* \b(\w+)\b)', sentence)
    new_list.append(bi_gram)
  return _pd.Series(new_list)


def _extract_tri_grams(x):
  new_list = []
  for idx,sentence in enumerate(x):
    tri_gram = _re.findall(r'(?=\b(\w+)\b\S* \b(\w+)\b\S* \b(\w+)\b)', sentence)
    new_list.append(tri_gram)
  return _pd.Series(new_list)
  

def _extract_n_gram(gram, x_train_whole_pre, this_size):
  if gram == 1:
      #do unigrams
      x_train_whole = _extract_uni_grams(x_train_whole_pre.iloc[0:this_size])
  if gram == 2:
      #do bigrams
      x_train_whole = _extract_bi_grams(x_train_whole_pre.iloc[0:this_size])
  if gram == 3:
    #do trigrams
    x_train_whole = _extract_tri_grams(x_train_whole_pre.iloc[0:this_size])
  return x_train_whole

In [0]:
#Takes training corpus and the indexes for a fold. 
#Returns (pd.series) the samples for the fold.
def separate_fold(x_train_whole, y_train_whole, train_idx, test_idx):
    x_train, x_test = x_train_whole.iloc[train_idx], x_train_whole.iloc[test_idx]
    y_train, y_test = y_train_whole.iloc[train_idx], y_train_whole.iloc[test_idx]
    #Split train and test
    x_train.reset_index(drop=True, inplace=True)
    x_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    #reset indexes to 0:(m-1)
    return x_train, y_train, x_test, y_test

#Takes (pd.series) training samples and their class
#Returns (dict) bag of words for sarcasm and bow for not sarcasm
def _make_bag_of_words(training_samples, training_samples_class):
    unique_grams = Counter()
    is_sarc_counts = Counter()
    not_sarc_counts = Counter()
    #thes dicts hold the counts of bi_grams 
    for idx,sentence in enumerate(training_samples):
        #for every training sentence in this fold
        for bg in sentence:
            #for every n-gram    
            unique_grams[bg] += 1
            if training_samples_class.iloc[idx] == 1:
                #if the sentence is sarcastic
                is_sarc_counts[bg] += 1
                #incrament that we've seen this n_gram as sarcastic
            else:
                #if sentence is not sarcastic
                not_sarc_counts[bg] += 1
                #incrament that we've seen this n_gram as not sarcastic
    return unique_grams, is_sarc_counts, not_sarc_counts

In [0]:
def _naive_bayes(unique_bigrams, is_sarc_counts, not_sarc_counts, num_training_samples, num_sarc_training_samples, x_test, alpha):
    num_unique_bg = len(unique_bigrams)
    #total number of unique bigrams
    num_sarc_bg = len(is_sarc_counts)
    #number of sarcastic bigrams
    num_not_sarc_bg = len(not_sarc_counts)
    #number of not sarcastic bigrams
    num_samples = num_training_samples
    num_sarc_samples = num_sarc_training_samples
    num_not_sarc_samples = num_samples - num_sarc_samples
    #just some precomputations..
    prior_prob_sarc = num_sarc_samples/num_samples
    prior_prob_not_sarc = num_not_sarc_samples/num_samples
    #the above lines compute the prior probabilites for both classes (sarc/not sarc)
    y_predict_test = []
    #holds the predicted class label
    for idx,sentence in enumerate(x_test):
        #for every sample
        prob_sarc = _log(prior_prob_sarc, 10)
        prob_not_sarc = _log(prior_prob_not_sarc, 10)
        for bg in sentence:
            prob_sarc = prob_sarc + _log((is_sarc_counts[bg] + alpha)/(num_sarc_bg + alpha*num_unique_bg), 10) 
            prob_not_sarc = prob_not_sarc + _log((not_sarc_counts[bg] + alpha)/(num_not_sarc_bg + alpha*num_unique_bg), 10)
        #after computing all bi_grams for this sentence
        if prob_sarc > prob_not_sarc:
            #if sarcastic is more likely than not sarcastic
            y_predict_test.append(1)
        else:
            #else predict not sarcastic
            y_predict_test.append(0)
    return y_predict_test


  
  
def _calculate_accuracy(y_actual, y_predict):
    num_right = 0
    total = len(y_actual)
    for idx, prediction in enumerate(y_predict):
        if prediction == y_actual.iloc[idx]:
            num_right += 1
    return ((num_right / total) * 100)
  

def _test_testing_data(x_train_whole, y_train_whole, x_test_whole, y_test_whole, alpha):
  corpus_counts, sarcastic_counts, not_sarcastic_counts = _make_bag_of_words(x_train_whole, y_train_whole)
  y_predict_whole = _naive_bayes(corpus_counts, sarcastic_counts, not_sarcastic_counts, len(x_train_whole), sum(y_train_whole), x_test_whole, alpha)
  accuracy = _calculate_accuracy(y_test_whole, y_predict_whole)
  return accuracy
  
  
  
#Takes list of tuples(alpha, acc) and range of alpha
#returns list of tuples(alpha, avg_acc)
#returns max alpha value
def analyze_folds(acc_list, alpha_list):
  avg_acc_list = []
  for alpha in alpha_list:
    this_avg = 0
    for tup in acc_list:
      if tup[0] == alpha:
        this_avg += tup[1]
    avg_acc_list.append((alpha,this_avg/5))
  max_alpha = max(avg_acc_list,key=itemgetter(1))[0]
  return avg_acc_list, max_alpha


#takes avg_acc_list
#saves chart to file
def save_alpha_plot_to_file(data, gram_str, size_str, alpha):
  filename = 'gdrive/My Drive/Data/data_out/alpha_train_' + gram_str + '_' + size_str + '.png'
  _title = size_str + " " + gram_str + " Alpha: " + str(alpha)
  _plt.scatter(*zip(*data))
  _plt.title(_title)
  _plt.xlabel("Alpha")
  _plt.ylabel("Accuracy")
  _plt.savefig(filename ,bbox_inches='tight')
  _plt.clf()
  
def save_size_plot_to_file(data, gram_str, size_str):
  filename = 'gdrive/My Drive/Data/data_out/SIZE_' + 'test' + '_' + gram_str + '.png'
  _title = 'Test Accuracy by Size: ' + gram_str
  _plt.scatter(*zip(*data))
  _plt.title(_title)
  _plt.xlabel("Number of Training Samples")
  _plt.ylabel("Accuracy")
  _plt.savefig(filename ,bbox_inches='tight')
  _plt.clf()

In [0]:
def run_size(x_train_whole, y_train_whole, x_test_whole, y_test_whole, gram_str, size_str):
  alpha_list = range(1,50)
  skf = StratifiedKFold(n_splits=5, shuffle=True)
  fold_num = 0
  alpha_accuracy_list = []
  #list of lists. sublists are accuracies for one fold by alpha value
  for train_idx, test_idx in skf.split(x_train_whole, y_train_whole):
      #for every fold
      #print('Fold', str(fold_num+1))
      x_train, y_train, x_test, y_test = separate_fold(x_train_whole, y_train_whole, train_idx, test_idx)
      #separate training data for this fold
      corpus_counts, sarcastic_counts, not_sarcastic_counts = _make_bag_of_words(x_train, y_train)
      #make bag of words for this fold
      for alpha in alpha_list:
          y_predict = _naive_bayes(corpus_counts, sarcastic_counts, not_sarcastic_counts, len(x_train), sum(y_train), x_test, alpha)
          #test training fold
          accuracy = _calculate_accuracy(y_test, y_predict)
          #print('\talpha:',str(round(alpha, 1)),'\taccuracy:',str(round(accuracy, 2)))
          alpha_accuracy_list.append((alpha, accuracy))
      fold_num += 1
  avg_acc_list, optimal_alpha = analyze_folds(alpha_accuracy_list, alpha_list)
  save_alpha_plot_to_file(avg_acc_list, gram_str, size_str, optimal_alpha)
  test_acc = _test_testing_data(x_train_whole, y_train_whole, x_test_whole, y_test_whole, optimal_alpha)
  return test_acc

In [0]:
#read in data and extract n-grams
x_train_whole_pre, y_train_whole_pre, x_test_whole_pre, y_test_whole =  _get_pickles()


sizes_list = [10000, 20000, 40000, 80000, 160000, 320000, 640000, len(x_train_whole_pre)]
grams_list = [1,2,3]
gram_str = ['unigram','bigram','trigram']
size_str = ['10k','20k','40k','80k','160k','320k','640k','810k']


for idx_g, gram in enumerate(grams_list):
    accuracy_by_size = []
    for idx_s, this_size in enumerate(sizes_list):
        print('currently on size ', size_str[idx_s], ' of ', gram_str[idx_g])
        x_train_whole = _extract_n_gram(gram, x_train_whole_pre, this_size)
        y_train_whole = y_train_whole_pre.iloc[0:this_size]
        x_test_whole = _extract_n_gram(gram, x_test_whole_pre, this_size)
        this_accuracy = run_size(x_train_whole, y_train_whole, x_test_whole, y_test_whole, gram_str[idx_g], size_str[idx_s])
        accuracy_by_size.append( (this_size, this_accuracy) )
    save_size_plot_to_file(accuracy_by_size, gram_str[idx_g], size_str[idx_s])